# Preprocessing function

In [1]:
import os

def find_files(directory, pattern):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(pattern):
                file_list.append(os.path.join(root, file))
    return file_list

directory = './'
files = find_files(directory, '.csv')

In [2]:
dir_path ='./'
dir_path_split = dir_path.split('/')
dir_path_split


['.', '']

In [1]:
import os
import re
import time
import datetime
import pandas as pd
import numpy as np

def find_files(directory, pattern):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(pattern):
                file_list.append(os.path.join(root, file))
    return file_list

def preprocess_file(dir_path):
    start_time = time.time()
    record = time.time()
    index = 0
    files = find_files(dir_path, '.csv')
    print(f'# of files : {len(files)}')
    
    #region [make forder for save tables]
    table_dir_path = os.path.join(dir_path, 'iot_tables')
    if not os.path.exists(table_dir_path):
        os.mkdir(table_dir_path)
    #endregion [make forder for save tables]
    
    #region [table columns] ############################################################
    
    #region [motor_col]
    motor_col = ['current_kw', 
                 'motor_spec_0',
                 'motor_spec_1', 
                 'motor_spec_2', 
                 'motor_spec_3']
    #endregion [motor_col]
    
    #region [c_meta_col]
    c_meta_col = ['record_id', 
                  'current_kw', 
                  'motor_spec_0',
                  'label_no',
                  'table_name',
                  'record_date',
                  'filename_dat',
                  'period_sec',
                  'sample_rate',
                  'data_length']
    #endregion [c_meta_col]
    
    #region [c_rms_col]
    c_rms_col = ['record_id',
                 'rms_1', 'rms_2', 'rms_3']
    #endregion [c_rms_col]
    
    #region [sensor_data_col]
    # sensor_data_col = ['reocrd_id']
    #endregion [sensor_data_col]
    
    #endregion [table columns]  ############################################################
    
    
    #region [make label table tsv file]
    label_table_file_name = 'label.tsv'
    label_table_path = os.path.join(table_dir_path, label_table_file_name)
    if not os.path.exists(label_table_path):
        
        label_table_data = {'label_no':[0, 1, 2, 3, 4],
                            'data_label':['normal', 
                                        'bad_bearing', 
                                        'rotro_imbalance',
                                        'shaft_misalignment',
                                        'belt_loose']}
        df_label = pd.DataFrame(label_table_data)
        df_label.to_csv(label_table_path, sep='\t', header=False, index=False)
    #endregion [make label table tsv file]


    #region [dir_path dir path level]
    dir_path_split = dir_path.split('/') # ex) dir_path : './' --> dir_path_split = ['.', '']
    path_lv = len(dir_path_split) - 2
    #endregion [dir_path dir path level]
    
    
    for file in files:
        df = pd.read_csv(file, sep='\t', header=None)
        file_split = file.split('/')
        
        if index==0: print(file_split)

            #region [file_split] ###############################
            
            # ['.',
            #  'current',
            #  '37kW',
            #  'L-PAHU-02S',
            #  '축정렬불량',
            #  'STFCB-20201012-0105-0138_20210204_160307_002.csv']
            
            #endregion [file_split] ###############################
        
        #region [current]
        # current_kw = float(file_split[2].replace('kW', ''))
        current_kw = float(file_split[path_lv + 2].replace('kW', ''))
        #endregion [current]
        
        #region [date]
        date_tmp = df.iloc[0, 0].split(',') # ex) ['Date', '2020-12-30 15:18:20']
        date = date_tmp[1]   # ex) '2020-12-30 15:18:20'     

        # for date error
        date = date_tmp[1].replace(':', '-')  # ex) '2020-12-30 15-18-20'
        #endregion [date]
        
        #region [file name]
        filename_dat_tmp = df.iloc[1, 0].split(',') # ex) ['Filename', 'STFCB-20201012-0105-0138_20210204_160304_002.dat']
        filename_dat_tmp = filename_dat_tmp[1].split('.')   # ex) ['STFCB-20201012-0105-0138_20210204_160304_002', 'dat']
        filename_dat = filename_dat_tmp[0] # ex) STFCB-20201012-0105-0138_20210204_160304_002
        #endregion [file name]
        
        #region [data label & label no]
        data_label_tmp = df.iloc[2, 0].split(',')   # ex) ['Data Label', '축정렬불량']
        label_kor = data_label_tmp[1]  # ex) 축정렬불량
        
        label_matching = {'정상':[0, 'normal'],
                          '베어링불량':[1, 'bad_bearing'],
                          '회전체불평형':[2, 'rotro_imbalance'],
                          '축정렬불량':[3, 'shaft_misalignment'],
                          '벨트느슨함':[4, 'belt_loose'],}
        
        label_no       = label_matching[label_kor][0]
        data_label  = label_matching[label_kor][1]
        #endregion [data label & label no]
        
        #region [motor spec]
        motor_spec_tmp = df.iloc[4, 0].split(',')   # ex) ['Motor Spec', 'L-PAHU-02S', '1770', '37', '71.1', '']
        motor_spec_0 = motor_spec_tmp[1]    # ex) L-PAHU-02S
        motor_spec_1 = motor_spec_tmp[2]
        motor_spec_2 = motor_spec_tmp[3]
        motor_spec_3 = motor_spec_tmp[4]
        #endregion [motor spec]

        #region [period sec]
        period_sec_tmp = df.iloc[5, 0].split(',')   # ex) ['Period', '1SEC']
        numbers = re.findall(r'\d+', period_sec_tmp[1]) # ex) ['1']
        period_sec = int(numbers[0]) if numbers else None # ex) 1
        #endregion [period sec]
        
        #region [sample rate]
        sample_rate_tmp = df.iloc[6, 0].split(',')  # ex) ['Sample Rate', '2000']
        sample_rate = sample_rate_tmp[1]    # ex) 2000
        #endregion [sample rate]

        #region [rms]
        rms_tmp = df.iloc[7, 0].split(',')  # ex) ['RMS', '34.72825', '35.11109', '34.39605', '']
        rms_1 = rms_tmp[1]  # ex) 34.72825
        rms_2 = rms_tmp[2]
        rms_3 = rms_tmp[3]
        #endregion [rms]

        #region [date length]
        data_length_tmp = df.iloc[8,0].split(',')   # ex) ['Data Length', '2000', '']
        data_length = data_length_tmp[1] # ex) 2000
        #endregion [date length]

        #region [data_0~3]
        df_data_0123 = df.iloc[9:]
        #endregion [data_0~3]
        
        #region [make table_name]
        current = file_split[path_lv + 1][0]   # ex) 'current' -> 'c'
        power = file_split[path_lv + 2].replace('.', 'p').replace('kW', '') # ex) '3.7kW' -> '3p7'
        device_name = file_split[path_lv + 3].replace('-', '_')  # ex) L-PAHU-02S
        
        sensor_data_tbl_name = f"{current}_{power}_{device_name}"
        #endregion [make table_name]

        meta_data = {
            # 'index': [int(index)],
            'record_id': [int(index)],
            
            # 'current_kw': [int(current_kw)],
            'current_kw': [float(current_kw)],
            'record_date': [str(date)],
            'filename_dat': [str(filename_dat)],
            'data_label': [str(data_label)],
            'label_no': [int(label_no)],
            'motor_spec_0': [str(motor_spec_0)],
            'motor_spec_1': [float(motor_spec_1)],
            'motor_spec_2': [float(motor_spec_2)],
            'motor_spec_3': [float(motor_spec_3)],
            'period_sec': [int(period_sec)],
            'sample_rate': [int(sample_rate)],
            'rms_1': [float(rms_1)],
            'rms_2': [float(rms_2)],
            'rms_3': [float(rms_3)],
            'data_length': [int(data_length)],
            
            'table_name':[str(sensor_data_tbl_name)]
            }
        
        
        #region [make dataframe per table]

                # tables_col = [motor_col, c_meta_col, c_rms_col, sensor_data_col]
                # table_name = ['motor', 'c_meta' 'c_rms', 'sensor_data']
        
        #region [make motor table tsv]
        motor_dict = {k: meta_data[k] for k in motor_col if k in meta_data}
        motor_tbl_path = os.path.join(table_dir_path, f'motor.tsv')
        
        df_motor = pd.DataFrame(motor_dict)
        
        
        
        if not os.path.exists(motor_tbl_path):
            # first file
            df_motor.to_csv(motor_tbl_path, sep='\t', header=None, index=False)
        else:
            # mode='a'
            df_motor.to_csv(motor_tbl_path, sep='\t', mode='a', header=None, index=False)
        
        #endregion [make motor table tsv]

        #region [c_meta table tsv]
        c_meta_tbl_path = os.path.join(table_dir_path, f'c_meta.tsv')
        c_meta_dict = {k: meta_data[k] for k in c_meta_col if k in meta_data}
        df_c_meta = pd.DataFrame(c_meta_dict)
        
        if not os.path.exists(c_meta_tbl_path):
            # first file
            df_c_meta.to_csv(c_meta_tbl_path, sep='\t', header=None, index=False)
        else:
            # mode='a'
            df_c_meta.to_csv(c_meta_tbl_path, sep='\t', mode='a', header=None, index=False)
        #endregion [c_meta table tsv]
        
        #region [c_rms table tsv]
        c_rms_tbl_path = os.path.join(table_dir_path, f'c_rms.tsv')
        c_rms_dict = {k: meta_data[k] for k in c_rms_col if k in meta_data}
        df_c_rms = pd.DataFrame(c_rms_dict)
        
        if not os.path.exists(c_rms_tbl_path):
            # first file
            df_c_rms.to_csv(c_rms_tbl_path, sep='\t', header=None, index=False)
        else:
            # mode='a'
            df_c_rms.to_csv(c_rms_tbl_path, sep='\t', mode='a', header=None, index=False)
        #endregion [c_rms table tsv]

        #region [v_meta table tsv]
        #endregion [v_meta table tsv]
        
        #region [v_rms table tsv]
        #endregion [v_rms table tsv]

        
        
        #region [sensor_data table] ex) c_2p2_[device name]
        df_data_0123 = df.iloc[9:][0].str.split(',', expand=True)   # df[9:11].columns --> Index([0], dtype='int64')
        # 원래 'combined' 컬럼 삭제
        df_data_0123.drop(4, axis=1, inplace=True)
        df_data_0123.reset_index(drop=True, inplace=True)
        record_id_repeat = np.repeat([meta_data['record_id']], len(df_data_0123), axis=0)
        record_id_df = pd.DataFrame(record_id_repeat)
        df_data_0123 = pd.concat((record_id_df, df_data_0123), axis=1)
        
        sensor_data_tbl_path = os.path.join(table_dir_path, f'{sensor_data_tbl_name}.tsv')
        if not os.path.exists(sensor_data_tbl_path):
            # first file
            df_data_0123.to_csv(sensor_data_tbl_path, sep='\t', header=None, index=False)
        else:
            # mode='a'
            df_data_0123.to_csv(sensor_data_tbl_path, sep='\t', mode='a', header=None, index=False)
        #endregion [sensor_data table] ex) c_2p2_[device name]

        # df_sensor_data
        # table_tsv_path = os.path.join(table_dir_path, f'c_rms.tsv')
        # c_rms_dict = {k: meta_data[k] for k in c_rms_col if k in meta_data}
        # df_c_rms = pd.DataFrame(c_rms_dict)
        
        # if not os.path.exists(table_dir_path):
        #     df_c_rms.to_csv(table_tsv_path, sep='\t', header=None, index=False)
        # else:
        #     df_c_rms.to_csv(table_tsv_path, sep='\t', mode='a', header=None, index=False)
        # #endregion [v_rms table tsv]
        index += 1
        
        # time
        term_scale = [100, 1000, 10000]
        if index < 1000: s = 0
        elif 1000 <= index < 10000: s = 1
        elif 10000 <= index: s = 2
        
        # if index % 100 == 0:
        if index % term_scale[s] == 0:
            print(f"index {index} out of {len(files)}")

            period = time.time() - record
            period = str(datetime.timedelta(seconds=period))
            period = period.split(".")[0] # second
            print(f"Record period: {period}")
            
            record = time.time()
            
            total = record - start_time
            total = str(datetime.timedelta(seconds=total))
            total = total.split(".")[0]
            print(f"Total time till now: {total}\n")

            print(f'[idx - {index}/{len(files)}] | Record period: {period} | ')
        
    #region [Remove motor table duplicate rows]
    df = pd.read_csv(motor_tbl_path, header=None, sep='\t')
    df = df.drop_duplicates()
    df.to_csv(motor_tbl_path, sep='\t', index=False, header=False)
    #endregion [Remove motor table duplicate rows]
        
    return 

# preprocess_file('./')
    

In [2]:
preprocess_file('./current')


# of files : 144730
['.', 'current', '15kW', 'L-CAHU-01S', '정상', 'STFCB-20200928-0105-0106_20210206_144007_002.csv']
index 100 out of 144730
Record period: 0:00:01
Total time till now: 0:00:01

[idx - 100/144730] | Record period: 0:00:01 | 
index 200 out of 144730
Record period: 0:00:01
Total time till now: 0:00:02

[idx - 200/144730] | Record period: 0:00:01 | 
index 300 out of 144730
Record period: 0:00:01
Total time till now: 0:00:03

[idx - 300/144730] | Record period: 0:00:01 | 
index 400 out of 144730
Record period: 0:00:01
Total time till now: 0:00:04

[idx - 400/144730] | Record period: 0:00:01 | 
index 500 out of 144730
Record period: 0:00:01
Total time till now: 0:00:05

[idx - 500/144730] | Record period: 0:00:01 | 
index 600 out of 144730
Record period: 0:00:01
Total time till now: 0:00:06

[idx - 600/144730] | Record period: 0:00:01 | 
index 700 out of 144730
Record period: 0:00:01
Total time till now: 0:00:07

[idx - 700/144730] | Record period: 0:00:01 | 
index 800 out o

In [ ]:
preprocess_file('./current')

# of files : 1156988
['.', 'current', '3.7kW', 'L-EF-01', '정상', 'STFCB-20201012-0105-0157_20210110_211406_002.csv']
index 100 out of 1156988
Record period: 0:00:04
Total time till now: 0:00:04

[idx - 100/1156988] | Record period: 0:00:04 | 
index 200 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:07

[idx - 200/1156988] | Record period: 0:00:02 | 
index 300 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:09

[idx - 300/1156988] | Record period: 0:00:02 | 
index 400 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:12

[idx - 400/1156988] | Record period: 0:00:02 | 
index 500 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:15

[idx - 500/1156988] | Record period: 0:00:02 | 
index 600 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:18

[idx - 600/1156988] | Record period: 0:00:02 | 
index 700 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:20

[idx - 700/1156988] | Record period: 0:00:02 | 
in

In [4]:
preprocess_file('./current')

# of files : 1156988
['.', 'current', '3.7kW', 'L-EF-01', '정상', 'STFCB-20201012-0105-0157_20210110_211406_002.csv']
index 100 out of 1156988
Record period: 0:00:04
Total time till now: 0:00:04

index 200 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:07

index 300 out of 1156988
Record period: 0:00:03
Total time till now: 0:00:10

index 400 out of 1156988
Record period: 0:00:03
Total time till now: 0:00:13

index 500 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:16

index 600 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:19

index 700 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:22

index 800 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:25

index 900 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:28

index 1000 out of 1156988
Record period: 0:00:02
Total time till now: 0:00:31

index 2000 out of 1156988
Record period: 0:00:30
Total time till now: 0:01:02

index 3000 out of 115698

# test ect

In [5]:
import pandas as pd
import os
import re
import time
import datetime

In [6]:
def generate_path_key(filepath):
    return filepath.replace('/', '_').replace('.', '_')

In [7]:
import os

def find_files(directory, pattern):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(pattern):
                file_list.append(os.path.join(root, file))
    return file_list

directory = './'
files = find_files(directory, '.csv')  # '.csv'는 파일 형식에 따라 변경

In [8]:
files

['./55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20201230_151607_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20210113_141407_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20210101_134707_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0132_20210204_125105_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20210123_181607_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20210202_132707_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20210127_101505_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20210123_171607_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0132_20210204_152007_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0132_20210206_134706_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20210101_091607_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0132_20210203_175805_002.csv',
 './55kw_sep/R-PAHU-04S/벨트느슨

In [9]:
files_split = files[0].split('/')
# files_split = files_split[1].split('\\')    # when rinux os don have to do this line(just for window os)
files_split

# ['.',
#  'current',
#  '37kW',
#  'L-PAHU-02S',
#  '축정렬불량',
#  'STFCB-20201012-0105-0138_20210204_160307_002.csv']

['.',
 '55kw_sep',
 'R-PAHU-04S',
 '벨트느슨함',
 'STFCB-20201012-0105-0125_20201230_151607_002.csv']

## DATE error test

In [11]:
file = files[0]
file

'./55kw_sep/R-PAHU-04S/벨트느슨함/STFCB-20201012-0105-0125_20201230_151607_002.csv'

In [13]:
#region [make forder for save tables]
table_dir_path = os.path.join(dir_path, 'iot_tables')
if not os.path.exists(table_dir_path):
    os.mkdir(table_dir_path)
#endregion [make forder for save tables]

#region [table columns] ############################################################

#region [motor_col]
motor_col = ['current_kw', 
             'motor_spec_0',
             'motor_spec_1', 
             'motor_spec_2', 
             'motor_spec_3']
#endregion [motor_col]

#region [c_meta_col]
c_meta_col = ['record_id', 
              'current_kw', 
              'motor_spec_0',
              'label_no',
              'table_name',
              'record_date',
              'filename_dat',
              'period_sec',
              'sample_rate',
              'data_length']
#endregion [c_meta_col]

#region [c_rms_col]
c_rms_col = ['record_id',
             'rms_1', 'rms_2', 'rms_3']
#endregion [c_rms_col]

#region [sensor_data_col]
# sensor_data_col = ['reocrd_id']	    # Integer	측정데이터 고유 ID (외래 키)
#endregion [sensor_data_col]

#endregion [table columns]  ############################################################


#region [make label table tsv file]
label_table_file_name = 'label.tsv'
label_table_path = os.path.join(table_dir_path, label_table_file_name)
if not os.path.exists(label_table_path):
    
    label_table_data = {'label_no':[0, 1, 2, 3, 4],
                        'data_label':['normal', 
                                    'bad_bearing', 
                                    'rotro_imbalance',
                                    'shaft_misalignment',
                                    'belt_loose']}
    df_label = pd.DataFrame(label_table_data)
    df_label.to_csv(label_table_path, sep='\t', header=False, index=False)    # column명에 대한 행 없이 tsv파일 저장
#endregion [make label table tsv file]


#region [dir_path 경로 레벨 카운트]
dir_path_split = dir_path.split('/') # ex) dir_path : './' --> dir_path_split = ['.', '']
path_lv = len(dir_path_split) - 2
#endregion [dir_path 경로 레벨 카운트]


df = pd.read_csv(file, sep='\t', header=None)
file_split = file.split('/')


    #region [file_split 구조 예시] ###############################
    
    # ['.',
    #  'current',
    #  '37kW',
    #  'L-PAHU-02S',
    #  '축정렬불량',
    #  'STFCB-20201012-0105-0138_20210204_160307_002.csv']
    
    #endregion [file_split 구조 예시] ###############################

# #region [current]
# # current_kw = float(file_split[2].replace('kW', ''))
# current_kw = float(file_split[path_lv + 2].replace('kW', ''))
# #endregion [current]



In [15]:
#region [date]
date_tmp = df.iloc[0, 0].split(',') # ex) ['Date', '2020-12-30 15:18:20']
print(date_tmp)
date = date_tmp[1]   # ex) '2020-12-30 15:18:20'     
print(date)
date = date_tmp[1].replace(':', '-')  # ex) '2020-12-30 15-18-20'
print(date)

#endregion [date]

['Date', '2020-12-30 15:18:20']
2020-12-30 15:18:20
2020-12-30 15-18-20


In [10]:
files_split = files[0].split('/')
files_split[1]

# 숫자만 추출
numbers = re.findall(r'\d+', files_split[1])

# 첫 번째 숫자만 출력하고 정수로 변환
number = int(numbers[0]) if numbers else None

print(number)  # 출력: 55


55


In [37]:
dir_path = './'
files = find_files(dir_path, '.csv')  # '.csv'는 파일 형식에 따라 변경
file = files[0]
index = 0

df = pd.read_csv(file, sep='\t', header=None)
file_split = file.split('/')
numbers = re.findall(r'\d+', file_split[2])
current_kw = int(numbers[0]) if numbers else None

date_tmp = df.iloc[0, 0].split(',')
date = date_tmp[1]


filename_dat_tmp = df.iloc[1, 0].split(',')
# print(filename_dat_tmp)
filename_dat_tmp = filename_dat_tmp[1].split('.')
# print(filename_dat_tmp)
filename_dat = filename_dat_tmp[0]
# print(filename_dat)


data_label_tmp = df.iloc[2, 0].split(',')
print(data_label_tmp)
# data_label_tmp = data_label_tmp[1]
label_kor = data_label_tmp[1]  # ex) 축정렬불량
print(label_kor)

label_matching = {'정상':[0, 'normal'],
                    '베어링불량':[1, 'bad_bearing'],
                    '회전체불평형':[2, 'rotro_imbalance'],
                    '축정렬불량':[3, 'shaft_misalignment'],
                    '벨트느슨함':[4, 'belt_loose'],}

label_no       = label_matching[label_kor][0]
# print(label_no)
data_label  = label_matching[label_kor][1]
# print(data_label)


motor_spec_tmp = df.iloc[4, 0].split(',')
# print(motor_spec_tmp)
motor_spec_0 = motor_spec_tmp[1]
# print(motor_spec_0)
motor_spec_1 = motor_spec_tmp[2]
motor_spec_2 = motor_spec_tmp[3]
motor_spec_3 = motor_spec_tmp[4]


period_sec_tmp = df.iloc[5, 0].split(',')
# print(period_sec_tmp)



# 숫자만 추출
numbers = re.findall(r'\d+', period_sec_tmp[1])
# print(numbers)
# 첫 번째 숫자만 출력하고 정수로 변환, ex) 55
period_sec = int(numbers[0]) if numbers else None
# print(period_sec)

# sample rate parsing
sample_rate_tmp = df.iloc[6, 0].split(',')
# print(sample_rate_tmp)
sample_rate = sample_rate_tmp[1]
# print(sample_rate)


rms_tmp = df.iloc[7, 0].split(',')
# print(rms_tmp)
rms_1 = rms_tmp[1]
# print(rms_1)
rms_2 = rms_tmp[2]
rms_3 = rms_tmp[3]


data_length_tmp = df.iloc[8,0].split(',')
# print(data_length_tmp)
data_length = data_length_tmp[1]
# print(data_length)

meta_data = {
    'index': [int(index)],
    'current_kw': [int(current_kw)],
    'date': [str(date)],
    'filename_dat': [str(filename_dat)],
    'data_label': [str(data_label)],
    'label_no': [int(label_no)],
    'motor_spec_0': [str(motor_spec_0)],
    'motor_spec_1': [float(motor_spec_1)],
    'motor_spec_2': [float(motor_spec_2)],
    'motor_spec_3': [float(motor_spec_3)],
    'period_sec': [int(period_sec)],
    'sample_rate': [int(sample_rate)],
    'rms_1': [float(rms_1)],
    'rms_2': [float(rms_2)],
    'rms_3': [float(rms_3)],
    'data_length': [int(data_length)]}

meta_data

['Data Label', '축정렬불량']
축정렬불량


{'index': [0],
 'current_kw': [37],
 'date': ['2021-02-04 16:04:10'],
 'filename_dat': ['STFCB-20201012-0105-0138_20210204_160304_002'],
 'data_label': ['shaft_misalignment'],
 'label_no': [3],
 'motor_spec_0': ['L-PAHU-02S'],
 'motor_spec_1': [1770.0],
 'motor_spec_2': [37.0],
 'motor_spec_3': [71.1],
 'period_sec': [1],
 'sample_rate': [2000],
 'rms_1': [34.72825],
 'rms_2': [35.11109],
 'rms_3': [34.39605],
 'data_length': [2000]}

In [38]:
df[9:11].columns

Index([0], dtype='int64')

In [41]:
df[9:].head()

,0
9,"0,-51.2099609375,33.79296875,17.8974609375,"
10,"0.0005,-52.53125,27.0263671875,25.865234375,"
11,"0.001,-54.453125,22.4619140625,32.3916015625,"
12,"0.0015,-49.6083984375,13.4130859375,36.35546875,"
13,"0.002,-49.087890625,7.9677734375,41.16015625,"


In [39]:
df_data_0123 = df.iloc[9:][0].str.split(',', expand=True)   # df[9:11].columns --> Index([0], dtype='int64')
df_data_0123.head()

,0,1,2,3,4
9,0,-51.2099609375,33.79296875,17.8974609375,
10,0.0005,-52.53125,27.0263671875,25.865234375,
11,0.001,-54.453125,22.4619140625,32.3916015625,
12,0.0015,-49.6083984375,13.4130859375,36.35546875,
13,0.002,-49.087890625,7.9677734375,41.16015625,


In [30]:
import pandas as pd

# 예제 데이터프레임 생성
data = {
    'combined': ['a,b,c', 'd,e,f', 'g,h,i']
}
df = pd.DataFrame(data)
print(df)
# 'combined' 컬럼을 쉼표를 기준으로 분할
split_df = df['combined'].str.split(',', expand=True)

# split_df = df.iloc[1:2]['combined'].str.split(',', expand=True)



# 분할된 결과를 새 컬럼으로 추가
split_df.columns = ['Col1', 'Col2', 'Col3']  # 새 컬럼 이름 지정
df = pd.concat([df, split_df], axis=1)

# 원래 'combined' 컬럼 삭제
df.drop('combined', axis=1, inplace=True)

# 결과 확인
print(df)

  combined
0    a,b,c
1    d,e,f
2    g,h,i
  Col1 Col2 Col3
0    a    b    c
1    d    e    f
2    g    h    i


#### path to sensordata table name


In [ ]:
def transform_path(path):
    # ex) path = './current/3.7kW/L-PAHU-02S/축정렬불량/STFCB-20201012-0105-0138_20210204_160307_002.csv'
    parts = path.split('/') # 경로를 '/' 기준으로 분리
    
    current = parts[1][0]   # ex) 'current'를 'c'로 변환
    power = parts[2].replace('.', 'p').replace('kW', '') # ex) '3.7kW'에서 소수점을 'p'로 변경하고 'kW' 제거
    device_name = parts[3]  # 기기명 ex) L-PAHU-02S
    
    sensor_data_table_name = f"{current}_{power}_{device_name}" # 변환된 부분들을 '_'로 연결하여 결과 반환
    return sensor_data_table_name

### label table

In [ ]:
# table name : DataLabel
    # LabelNo	Integer	라벨 번호 (고장 유형을 숫자로 표시) (기본키)
    # data_label	String	데이터 라벨 (정상, 고장 등)

# LabelNo	Integer	라벨 번호 (고장 유형을 숫자로 표시) (기본키)
# csv 헤더 파일에서 LabelNo가 있는 행 찾기 --> 숫자 추출 저장

# data_label	String	데이터 라벨 (정상, 고장 등)
# LabelNo의 숫자를 data_label_description 딕셔너리의 key로 사용해서 value를 저장
data_label_description = {0:'dd',
                          1:'ddd',
                          2:'dddd',
                          3:'ddddd',
                          4:'dddddd'}




### motor table

In [ ]:
# table name : Motor
    # 필드명	데이터 유형	설명
    # current_kw	Float	모터 용량 (kW)
    # motor_spec_0	String	모터 기기명
    # motor_spec_1	String	모터 rpm
    # motor_spec_2	Integer	정격출력
    # motor_spec_3	Float	정격전류


### c_meta table (Current_Measurement)

In [4]:
# table name : Current_Measurement
    # 필드명	데이터 유형	설명
    # reocrd_id 	    Integer	측정데이터 고유 ID (기본 키)
    # current_kw	    Float	모터 용량 (kW) (외래키)
    # motor_spec_0	    String	모터 기기명
    # label_no	        Integer	라벨 번호 (외래 키)
    # table_name	    string	ex) Current_2p2kw_[기기명]
    # record_date	    DateTime	데이터가 기록된 날짜와 시간
    # filename_dat	    String	데이터 파일 이름
    # period_sec	    Integer	샘플링 주기 (초)
    # sample_rate	    Integer	샘플링 레이트 (Hz)
    # data_length	    Integer	데이터 길이


### c_rms, v_rms table  --> e.g., Current_RMS, Vibration_RMS

In [ ]:
# table name : e.g., Current_RMS, Vibration_RMS
    # reocrd_id 	Integer	측정데이터 고유 ID (외래키)
    # rms_1	        Float	X축-RMS
    # rms_2	        Float	Y축-RMS
    # rms_3	        Float	Z축-RMS


### Sensordata table

In [ ]:
# table name : [c or v]_[CurrentKW]_[motor_spec_0] 
    # --> ex) current이고 15kW이고 [기기명]인 경우에는 c_15_[기기명]
    # --> ex) current이고 2.2kW이고 [기기명]인 경우에는 c_2p2_[기기명]

    # 필드명	데이터 유형	설명
    # reocrd_id	    Integer	측정데이터 고유 ID (기본 키)
    # current_kw	Float	모터 용량 (kW) (외래키)
    # motor_spec_0	String	모터 기기명
    # label_no	    Integer	라벨 번호 (외래 키)
    # table_name	string	ex) Current_2p2kw_[기기명]
    # record_date	DateTime	데이터가 기록된 날짜와 시간
    # filename_dat	String	데이터 파일 이름
    # period_sec	Integer	샘플링 주기 (초)
    # sample_rate	Integer	샘플링 레이트 (Hz)
    # data_length	Integer	데이터 길이
